In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fuzzywuzzy


In [4]:
import nltk
import random
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from fuzzywuzzy import fuzz

# Download necessary NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define intents and responses
intents = {
    'greeting': {
        'patterns': ['hi','hello','hey','greetings','what\'s up','howdy','good morning','good evening','how are you?'],
        'responses': [
            'Hello! How can I help you today?'
            'Hi there! What would you like to know?',
            'Greetings! I\'m here to assist you with any questions.',
            'Good day! How can I assist you?',
            'Hey! What’s on your mind?',
            'I\'m doing well, thank you! How about you?'
        ]
    },
    'job_openings': {
        'patterns': [
            'job openings', 'available positions', 'job listings',
            'career opportunities', 'current vacancies', 'open positions',
            'job search', 'job opportunities', 'job application',
            'hiring now', 'job vacancies','where can i find jobs',
        ],
        'responses': [
            'To find job openings, I recommend checking popular job boards like LinkedIn, Indeed, or Glassdoor.',
            'You can also visit company websites for their career pages to find available positions.',
            'Don\'t forget to network! Sometimes the best opportunities come from connections.',
            'You can set up job alerts on job boards to receive notifications for new openings that match your criteria.',
            'Consider using recruitment agencies that specialize in your field for more tailored job opportunities.',
            'Networking events and job fairs can also be great places to discover job openings.'

        ]
    },
    'skill_requirements': {
        'patterns': [
            'skill requirements', 'required skills', 'what skills do I need',
            'skill set', 'skills needed', 'skills for jobs',
            'skills needed for [specific job]', 'what skills are in demand',
            'essential skills for [specific field]'

        ],
        'responses': [
            'Skill requirements vary depending on the job. Common skills include communication, problem-solving, and teamwork.',
            'For specific roles, you might need technical skills like programming languages or data analysis.',
            'Consider researching job descriptions in your field to identify key skills.',
            'In addition to technical skills, soft skills like adaptability and emotional intelligence are becoming increasingly important.',
            'Look for online resources or industry reports that highlight trending skills in your field.',
            'Certifications can also enhance your qualifications for specific roles.'
        ]
    },
    'resume_building': {
        'patterns': [
            'resume building', 'how to write a resume', 'resume tips',
            'CV advice', 'creating a resume', 'resume format',
            'resume examples', 'how to update my resume',
            'resume mistakes to avoid'
        ],
        'responses': [
            'When building a resume, focus on your relevant skills and experiences.',
            'Use action verbs, quantify your achievements, and tailor your resume to the job description.',
            'Keep it concise and well-formatted. Consider using templates available online.',
            'Make sure to customize your resume for each job application to highlight the most relevant experience.',
            'Avoid common mistakes like typos or using an unprofessional email address.'
        ]
    },
    'skill_improvement': {
        'patterns': [
            'improve skills', 'skill development', 'learn new skills',
            'upgrade skillset', 'how to learn skills', 'best ways to learn skills',
            'resources for skill development', 'how to practice skills',
            'mentorship opportunities'
        ],
        'responses': [
            'To improve your skills, consider taking online courses, attending workshops, or working on personal projects.',
            'Platforms like Coursera, edX, and Udemy offer a wide range of courses to enhance your skillset.',
            'Joining community groups or forums can also provide valuable resources and support.',
            'Consider finding a mentor in your field who can provide guidance and resources for skill development.',
            'Participating in hackathons or competitions can also be a fun way to improve your skills while networking.'
        ]
    },
    'interview_tips': {
        'patterns': [
            'interview tips', 'how to prepare for an interview',
            'interview advice', 'succeed in interviews',
            'common interview questions', 'how to answer behavioral questions',
            'what to wear to an interview', 'follow-up after interview'
        ],
        'responses': [
            'Research the company and the role you\'re applying for .',
            'Practice common interview questions and prepare your own questions to ask the interviewer.',
            'dress appropriately and arrive on time. Confidence and good communication are key!',
            'Prepare examples of your past experiences that demonstrate your skills and achievements.',
            'Sending a thank-you email after the interview can leave a positive impression on the interviewer.',
            ' wear a business professional attire'
        ]
    },
    'salary_negotiation': {
        'patterns': [
            'salary negotiation', 'how to negotiate salary',
            'salary tips', 'negotiating a raise',
            'salary expectations', 'what to say during salary negotiation'
        ],
        'responses': [
            'Research the market rate for your role to determine a fair salary range.',
            'Be confident and assertive during the negotiation, but also be open to compromise.',
            'Highlight your achievements and the value you bring to the company.',
            'Consider factors beyond salary, such as benefits, vacation time, or professional development opportunities.',
            'Be prepared to explain why you\'re worth the salary you\'re asking for.'
        ]
    },
    'course_recommendation': {
        'patterns': ['recommend a course', 'suggest a course', 'what course should I take', 'course recommendation'],
        'responses': ['Based on your interests, I\'d recommend the following course: {course_name}. You can find it here: {course_url}']
    },
    'skill_based_course': {
        'patterns': ['course for {skill}', 'learn {skill}', 'improve my {skill}'],
        'responses': ['For {skill}, I\'d recommend the course: {course_name}. You can find it here: {course_url}']
    }
}


# Function to tokenize and lemmatize input text
def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

# Function to match user input with intents
def match_intent(user_input):
    tokens = tokenize_and_lemmatize(user_input)
    for intent, data in intents.items():
        for pattern in data['patterns']:
            pattern_tokens = tokenize_and_lemmatize(pattern)
            if all(token in tokens for token in pattern_tokens):
                return intent
    return None

# Function to find a course based on skill or any other input
def find_course(input_text):
    matching_courses = []
    input_text = input_text.lower()

    # Check each course for skill matches
    for course_id, course in courses.items():
        # Convert skills to lowercase for case-insensitive matching
        skills = [skill.lower().strip() for skill in course['job_skills']]

        # Calculate skill match similarity
        skill_match = False
        for skill in skills:
            # Check if input text contains any of the skills
            if (input_text in skill) or (skill in input_text) or (fuzz.ratio(input_text, skill) > 70):
                skill_match = True
                break

        if skill_match:
            similarity = fuzz.ratio(input_text, course['course_name'].lower()) * 0.3 + \
                        max([fuzz.ratio(input_text, skill.lower()) for skill in course['job_skills']]) * 0.7

            matching_courses.append({
                'course_name': course['course_name'],
                'course_url': course['course_url'],
                'relevance': similarity/100,
                'skills': course['job_skills']
            })

    # Sort matches by relevance score
    matching_courses.sort(key=lambda x: x['relevance'], reverse=True)

    # Return top 3 most relevant matches
    return matching_courses[:3]

def respond(user_input):
    intent = match_intent(user_input)

    if intent in ['course_recommendation', 'skill_based_course'] or 'course' in user_input.lower():
        matching_courses = find_course(user_input)
        if matching_courses:
            response = "Here are some relevant course recommendations based on your skills interest:\n\n"
            for i, course in enumerate(matching_courses, 1):
                response += f"{i}. {course['course_name']}\n"
                response += f"   Relevance Score: {course['relevance']:.2f}\n"
                response += f"   Course Link: {course['course_url']}\n"
                response += f"   Related Skills: {', '.join(course['skills'])}\n\n"
            return response
        else:
            return "I couldn't find any courses matching your criteria. Try searching for a different skill or topic."
    elif intent:
        return random.choice(intents[intent]['responses'])
    else:
        return "I'm not sure what you're looking for. Try asking about specific skills or courses you're interested in."
# Load courses data
courses = {}
with open('/content/job_dataset.csv', 'r') as file:
    reader = csv.reader(file)
    headers = next(reader)  # Read the header row
    for row in reader:
        course_id = row[0]
        courses[course_id] = {
            'job_skills': row[7].split(','),
            'job_type': row[5],
            'job_category': row[6],
            'job_roles': row[8].split(','),
            'course_name': row[9],
            'course_url': row[10]
        }


# Run the chatbot
if __name__ == "__main__":
    print("Chatbot: Hello! I'm here to help you with job-related queries and course recommendations. How can I assist you today?")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Chatbot: Goodbye! Have a great day!")
            break

        response = respond(user_input)
        print("Chatbot:", response)

Chatbot: Hello! I'm here to help you with job-related queries and course recommendations. How can I assist you today?
You: hi
Chatbot: Good day! How can I assist you?
You: what to wear for an interview
Chatbot:  wear a business professional attire
You: how to attend an interview
Chatbot: I'm not sure what you're looking for. Try asking about specific skills or courses you're interested in.
You: can you recommend a course for Design
Chatbot: I couldn't find any courses matching your criteria. Try searching for a different skill or topic.
You: can you recommend a course for Machine Learning
Chatbot: Here are some relevant course recommendations based on your skills interest:

1. Structuring Machine Learning Projects
   Relevance Score: 0.50
   Course Link: https://www.coursera.org/learn/machine-learning-projects
   Related Skills: Machine Learning, data analysis, Python

2. Uncommon Sense Teaching
   Relevance Score: 0.49
   Course Link: https://www.coursera.org/learn/uncommon-sense-teac